**Predicting Functional Threshold Power (FTP)**

In [27]:
import os
import pandas as pd
import strava_export as strava

athletes = pd.read_csv(
    os.path.join(strava.pro_dir, 'athletes.csv'),
    index_col = 'id'
)

rides = pd.read_csv(
    os.path.join(strava.pro_dir, 'rides.csv'),
    index_col = ['id', 'timestamp']
)

rc = rides.groupby('id').size()
athletes = athletes.merge(rc.to_frame(), left_index=True, right_index=True)
print(athletes)

                               email firstname lastname sex  weight    0
id                                                                      
2163639        alun.hanford@live.com      Alun  Hanford   M    68.0  523
4917647  jamesrobertkinley@gmail.com     James   Kinley   M    68.0  261
